In [20]:
!pip install datasets evaluate transformers[sentencepiece] python-docx

In [21]:
!pip install pytorch-lightning wandb onnx

In [22]:
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification, BertModel
from transformers import BertConfig, BertPreTrainedModel
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset
import json
import pytorch_lightning as pl
from torchmetrics import Metric
import wandb

In [23]:
!wandb login

wandb: Currently logged in as: prince_. Use `wandb login --relogin` to force relogin


In [24]:
# Read the JSON file
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

In [25]:
path = "/content/drive/MyDrive/model/training_set.json"
with open(path, 'r') as f:
    test_dataset = json.load(f)

In [26]:
text, intent, ner = [], [], []
for i in dataset:
    text.append(i['text'])
    intent.append(i['intent'])
    ner.append(i['entities'].split())

In [27]:
test_text, test_intent, test_ner = [], [], []
for i in test_dataset:
    test_text.append(i['text'])
    test_intent.append(i['intent'])
    test_ner.append(i['entities'].split())

In [28]:
unique_intents = set(intent)
num_intent_labels = len(unique_intents)

unique_intents, num_intent_labels

({"'Schedule Appointment'",
  "'Schedule Meeting'",
  "'Set Alarm'",
  "'Set Reminder'",
  "'Set Timer'"},
 5)

In [29]:
one_dimensional_ner = [tag for subset in ner for tag in subset ]
unique_ner = set(one_dimensional_ner)
num_ner_labels = len(unique_ner)
unique_ner, num_ner_labels

({'B-DATE',
  'B-DUR',
  'B-TASK',
  'B-TIME',
  'I-DATE',
  'I-DUR',
  'I-TASK',
  'I-TIME',
  'O'},
 9)

In [30]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased')

In [31]:
labels_to_ids_ner = {
    'O': 0,
    'B-DATE': 1,
    'I-DATE': 2,
    'B-TIME': 3,
    'I-TIME': 4,
    'B-TASK': 5,
    'I-TASK': 6,
    'B-DUR': 7,
    'I-DUR': 8
    }

ids_to_labels_ner = {v: k for k, v in labels_to_ids_ner.items()}
ids_to_labels_ner

{0: 'O',
 1: 'B-DATE',
 2: 'I-DATE',
 3: 'B-TIME',
 4: 'I-TIME',
 5: 'B-TASK',
 6: 'I-TASK',
 7: 'B-DUR',
 8: 'I-DUR'}

In [32]:
labels_to_ids_intent = {
    "'Schedule Appointment'": 0,
    "'Schedule Meeting'": 1,
    "'Set Alarm'": 2,
    "'Set Reminder'": 3,
    "'Set Timer'": 4
}

ids_to_labels_intent = {v: k for k, v in labels_to_ids_intent.items()}
ids_to_labels_intent

{0: "'Schedule Appointment'",
 1: "'Schedule Meeting'",
 2: "'Set Alarm'",
 3: "'Set Reminder'",
 4: "'Set Timer'"}

In [33]:
class dataset(Dataset):
    def __init__(self, text, intent, ner, tokenizer, max_len=128):
        self.len = len(text)
        self.text = text
        self.intent = intent
        self.ner = ner
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence, ner label, and intent_label
        sentence = self.text[index].strip()
        intent_label = self.intent[index].strip()
        ner_labels = self.ner[index]

        # step 2: use tokenizer to encode a sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" which highlights where each token starts and ends
        encoding = self.tokenizer(
            sentence,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        # step 3: create ner token labels only for first word pieces of each tokenized word
        tokenized_ner_labels = [labels_to_ids_ner[label] for label in ner_labels]
        # create an empty array of -100 of length max_length
        encoded_ner_labels = np.ones(len(encoding['offset_mapping']), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        prev = -1
        for idx, mapping in enumerate(encoding['offset_mapping']):
            if mapping[0] == mapping[1] == 0:
                continue
            if mapping[0] != prev:
                # overwrite label
                encoded_ner_labels[idx] = tokenized_ner_labels[i]
                prev = mapping[1]
                i += 1
            else:
                prev = mapping[1]

        # create intent token labels
        tokenized_intent_label = labels_to_ids_intent[intent_label]

        # step 4: turn everything into Pytorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['ner_labels'] = torch.as_tensor(encoded_ner_labels)
        item['intent_labels'] = torch.as_tensor(tokenized_intent_label)

        return item

    def __len__(self):
        return self.len

In [34]:
training_set = dataset(text, intent, ner, tokenizer)
validation_set = dataset(test_text, test_intent, test_ner, tokenizer)

In [35]:
class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state('total', default=torch.tensor(0), dist_reduce_fx='sum')
        self.add_state('correct', default=torch.tensor(0), dist_reduce_fx='sum')

    def update(self, logits, labels, num_labels):

        flattened_targets = labels.view(-1) # shape (batch_size, sequence_len)
        active_logits = logits.view(-1, num_labels) # shape (batch_size * sequence_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * sequence_len)

        # compute accuracy only at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, sequnce_len)
        ac_labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        self.correct += torch.sum(ac_labels == predictions)
        self.total += torch.numel(ac_labels)

    def compute(self):
        return self.correct.float() / self.total.float()


In [36]:
class DataModule(pl.LightningDataModule):
    def __init__(self, training_set, validation_set):
        super().__init__()
        self.training_set = training_set
        self.validation_set = validation_set

    def setup(self, stage):
        self.train_ds = self.training_set
        self.val_ds = self.validation_set

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=1, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=1, shuffle=False)

In [37]:
class MultiTaskBertModel(pl.LightningModule):

    """
    Multi-task Bert model for Named Entity Recognition (NER) and Intent Classification

    Args:
        config (BertConfig): Bert model configuration.
        num_ner_labels (int): The number of labels for NER task.
        num_intent_labels (int): The number of labels for Intent Classification task.
    """

    def __init__(self, config, num_ner_labels, num_intent_labels):
        super().__init__()

        self.num_ner_labels = num_ner_labels
        self.num_intent_labels = num_intent_labels

        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.model = BertModel(config=config)

        self.ner_classifier = torch.nn.Linear(config.hidden_size, self.num_ner_labels)
        self.intent_classifier = torch.nn.Linear(config.hidden_size, self.num_intent_labels)

        # log hyperparameters
        self.save_hyperparameters()

        self.accuracy = MyAccuracy()

    def forward(self, input_ids=None, attention_mask=None):

        """
        Perform a forward pass through Multi-task Bert model.

        Args:
            input_ids (torch.Tensor): Input token IDs.
            attention_mask (torch.Tensor): Attention mask for input tokens.
            ner_labels (torch.Tensor): Labels for NER task.
            intent_labels (torch.Tensor): Labels for Intent Classification task.

        Returns:
            Tuple[torch.Tensor,torch.Tensor,torch.Tensor,torch.Tensor]: NER loss, NER logits, Intent loss, Intent logits.

        Raises:
            ValueError: If ner_labels or intent_labels were not provided.
        """

        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        ner_logits = self.ner_classifier(sequence_output)

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        intent_logits = self.intent_classifier(pooled_output)

        return ner_logits, intent_logits

    def training_step(self: pl.LightningModule, batch, batch_idx: int):
        loss, ner_logits, intent_logits, ner_labels, intent_labels = self._common_step(batch, batch_idx)
        accuracy_ner = self.accuracy(ner_logits, ner_labels, num_ner_labels)
        accuracy_intent = self.accuracy(intent_logits, intent_labels, num_intent_labels)
        self.log_dict({'training_loss': loss, 'ner_accuracy': accuracy_ner, 'intent_accuracy': accuracy_intent},
                      on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def on_validation_epoch_start(self):
        self.validation_step_outputs_ner = []
        self.validation_step_outputs_intent = []

    def validation_step(self, batch, batch_idx):
        loss, ner_logits, intent_logits, ner_labels, intent_labels = self._common_step(batch, batch_idx)
        # self.log('val_loss', loss)
        accuracy_ner = self.accuracy(ner_logits, ner_labels, num_ner_labels)
        accuracy_intent = self.accuracy(intent_logits, intent_labels, num_intent_labels)
        self.log_dict({'validation_loss': loss, 'val_ner_accuracy': accuracy_ner, 'val_intent_accuracy': accuracy_intent},
                      on_step=False, on_epoch=True, prog_bar=True)

        self.validation_step_outputs_ner.append(ner_logits)
        self.validation_step_outputs_intent.append(intent_logits)
        return loss

    def on_validation_epoch_end(self):
        validation_step_outputs_ner = self.validation_step_outputs_ner
        validation_step_outputs_intent = self.validation_step_outputs_intent

        dummy_input = torch.zeros((1, 128), device=self.device, dtype=torch.long)
        model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
        torch.onnx.export(self, dummy_input, model_filename)
        artifact = wandb.Artifact(name="model.ckpt", type="model")
        artifact.add_file(model_filename)
        self.logger.experiment.log_artifact(artifact)

        flattened_logits_ner = torch.flatten(torch.cat(validation_step_outputs_ner))
        flattened_logits_intent = torch.flatten(torch.cat(validation_step_outputs_intent))
        self.logger.experiment.log(
            {"valid/ner_logits": wandb.Histogram(flattened_logits_ner.to('cpu')),
             "valid/intent_logits": wandb.Histogram(flattened_logits_intent.to('cpu')),
             "global_step": self.global_step}
        )

    def _common_step(self, batch, batch_idx):
        ids = batch['input_ids']
        mask = batch['attention_mask']
        ner_labels = batch['ner_labels']
        intent_labels = batch['intent_labels']

        ner_logits, intent_logits = self.forward(ids, mask)

        criterion = torch.nn.CrossEntropyLoss()

        ner_loss = criterion(ner_logits.view(-1, self.num_ner_labels), ner_labels.view(-1))
        intent_loss = criterion(intent_logits.view(-1, self.num_intent_labels), intent_labels.view(-1))

        loss = ner_loss + intent_loss
        return loss, ner_logits, intent_logits, ner_labels, intent_labels

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        return optimizer


In [38]:
wandb_logger = pl.loggers.WandbLogger(project='lit-wandb')

# Trainer

In [39]:
model = MultiTaskBertModel(config, num_ner_labels, num_intent_labels)
dm = DataModule(training_set, validation_set)
trainer = pl.Trainer(
    logger = wandb_logger,
    log_every_n_steps=40,
    max_epochs=7,
    deterministic=True)
trainer.fit(model, dm)
trainer.validate(model, dm)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: Currently logged in as: prince_. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type       | Params
-------------------------------------------------
0 | dropout           | Dropout    | 0     
1 | model             | BertModel  | 109 M 
2 | ner_classifier    | Linear     | 6.9 K 
3 | intent_classifier | Linear     | 3.8 K 
4 | accuracy          | MyAccuracy | 0     
-------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.972   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=40). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=8` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: torch.Size([1, 128, 9]), intent logits: torch.Size([1, 5])
Ids: torch.Size([1, 128]), mask: torch.Size([1, 128]), ner: torch.Size([1, 128]), intent:torch.Size([1])
ner logits: tor

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    val_intent_accuracy    │    0.7083333134651184     │
│     val_ner_accuracy      │    0.8226388096809387     │
│      validation_loss      │    1.3291294574737549     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇█
global_step,▁▂▃▄▅▅▆▇██
intent_accuracy,▁▂▃▅▆▆▇█
ner_accuracy,▁▂▅▆▇███
trainer/global_step,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████
training_loss,█▆▅▄▃▂▁▁
val_intent_accuracy,▁▂▃▆▅▇█▆▆
val_ner_accuracy,▁▄▅▆▇████
validation_loss,█▆▄▃▂▂▁▂▂
epoch,8
global_step,280
